In [47]:
import torchvision
from torchvision import transforms
from torch.utils.data import Subset
import torch
from torch.utils.data import DataLoader
import torch.nn as nn

In [48]:
image_path = './'
transform = transforms.Compose([
    transforms.ToTensor()
    ])

mnist_dataset = torchvision.datasets.FashionMNIST(
    root=image_path, train=True,
    transform=transform, download=True
    )

mnist_train_dataset = Subset(mnist_dataset, torch.arange(3000))
mnist_test_dataset = Subset(mnist_dataset, torch.arange(3000, 4000))

In [49]:
print(f"Tenemos un tamaño en train de {len(mnist_train_dataset)} y en test de {len(mnist_test_dataset)}.")
image, label = mnist_train_dataset[0]
print(f"El tamaño de la imagen es de {image.shape[0]}x{image.shape[1]}x{image.shape[2]}")

Tenemos un tamaño en train de 3000 y en test de 1000.
El tamaño de la imagen es de 1x28x28


In [50]:
# Creamos los batch
batch_size = 64
torch.manual_seed(1) # Imporante poner la semilla 
train_dl = DataLoader(mnist_train_dataset,
                      batch_size,
                      shuffle=True)

test_dl = DataLoader(mnist_test_dataset,
                      batch_size,
                      shuffle=False)

In [51]:
# Creamos la secuencia de pasos
model = nn.Sequential()

# _______________________________________________
# Capa convolucion 1
model.add_module(
    'convLayer1',
    nn.Conv2d(
     in_channels = 1, out_channels = 32,
     kernel_size = 5, padding = 2
    )
)
# Capa funcion de activacion
model.add_module('relu1',nn.ReLU())

# Pooling 
model.add_module('pool1', nn.MaxPool2d(kernel_size=2))
# _______________________________________________
# Capa convolucion 2
model.add_module(
    'conv2',
    nn.Conv2d(
        in_channels = 32, out_channels = 64,
        kernel_size = 5, padding = 2
    )
)

# Capa funcion de activacion
model.add_module('relu2',nn.ReLU())

# Capa pooling 
model.add_module('pool2', nn.MaxPool2d(kernel_size = 2))
# _______________________________________________



In [52]:
# Usamos una imagen de prueba para ver el shape de la salida
x = torch.ones((4,1,28,28)) # Cuatro imagenes de 1x28x28 de prueba
model(x).shape # Tamaño de la salida -> 4x64x7x7

torch.Size([4, 64, 7, 7])

In [53]:
# Veamos el tamaño del vector tras el flattern
model.add_module('flattern',nn.Flatten())
model(x).shape # Tamaño del vector -> 4x3136 ; 3136 = 64x7x7

torch.Size([4, 3136])

In [54]:
# Terminamos el modelo metiendo la red neuronal 

# Capa 1
model.add_module('fc1', nn.Linear(3136, 1024))
# Funcion activacion 1
model.add_module('relu3', nn.ReLU())
# Metemos droput para mejorar la generalizacion
model.add_module('drop_out', nn.Dropout())
# Capa salida
model.add_module('fc2', nn.Linear(1024, 10)) 

In [55]:
# Asignamos la funcion de coste y el optimizador
loss_fn = nn.CrossEntropyLoss() # Esta ya contiene la softmax, se le pasan los logit
optimizers = [torch.optim.Adam(model.parameters(), lr = 0.001)]
optimizers.append(torch.optim.SGD(model.parameters(), lr = 0.001))

In [ ]:
# ENTRENAMIENTO
def train(model, n_epochs, train_dl, test_dl):
    opts = ['Adam','Schocastic Gradient Descent']
    i = 0
    for opt in optimizers:
        print(f"\nIntento con {opts[i]}")
        i += 1
        # Variables para almacenar los resultados de cada epoca
            # Train
        loss_hist_train = [0] * n_epochs 
        accuracy_hist_train = [0] * n_epochs 
            # Test
        loss_hist_test = [0] * n_epochs
        accuracy_hist_test = [0] * n_epochs
        
        # Iteramos por epocas
        for epoch in range(n_epochs):
            model.train() # Modo entrenamiento
            for x_batch, y_batch in train_dl:
                prediction = model(x_batch) # Predecimos
                loss = loss_fn(prediction, y_batch) # Calculamos el loss (este contiene el softmax)
                loss.backward() # Calculamos los gradientes
                opt.step() # Actualizamos los pesos con el optimizador
                opt.zero_grad() # Vaciamos los gradientes

                loss_hist_train[epoch] += loss.item() * y_batch.size(0) # Calculamos la perdida
                
                is_correct = ( # True, false, true, false...
                    torch.argmax(prediction, dim=1) == y_batch
                ).float()
                accuracy_hist_train[epoch] += is_correct.sum()  # Calculamos las predicciones correctas
            
            loss_hist_train[epoch] /= len(train_dl.dataset) # Dividimos el numero de predicciones correctas entre el numero de observaciones 
            accuracy_hist_train[epoch] /= len(train_dl.dataset)
        
            
            model.eval() # Modo evaluacion -> Test
            with torch.no_grad():
                for x_batch, y_batch in test_dl:
                    prediction = model(x_batch) # Calculamos la prediccion
                    loss = loss_fn(prediction, y_batch) # Calulamos la funcion de coste
                    
                    loss_hist_test[epoch] += loss.item() * y_batch.size(0)
                    
                    is_correct = (
                        torch.argmax(prediction, dim=1) == y_batch
                    ).float()
                    accuracy_hist_test[epoch] += is_correct.sum()
                
            loss_hist_test[epoch] /= len(test_dl.dataset) # Dividimos el numero de predicciones correctas entre el numero de observaciones 
            accuracy_hist_test[epoch] /= len(test_dl.dataset)

            
            print(f'Epoca {epoch+1} accuracy: '
                f'{accuracy_hist_train[epoch]}'
                f'\n\t Test accuracy:{accuracy_hist_test[epoch]}')
     

In [65]:
torch.manual_seed(1)
num_epochs = 20
train(model, num_epochs, train_dl, test_dl)

Intento con Adam
Epoca 1 accuracy: 0.9923333525657654
	 Test accuracy:0.8700000047683716
Epoca 2 accuracy: 0.9973333477973938
	 Test accuracy:0.8669999837875366
Epoca 3 accuracy: 0.9990000128746033
	 Test accuracy:0.8709999918937683
Epoca 4 accuracy: 0.9953333139419556
	 Test accuracy:0.8740000128746033
Epoca 5 accuracy: 0.996666669845581
	 Test accuracy:0.8650000095367432
Epoca 6 accuracy: 0.9940000176429749
	 Test accuracy:0.8690000176429749
Epoca 7 accuracy: 0.9986666440963745
	 Test accuracy:0.875
Epoca 8 accuracy: 0.9983333349227905
	 Test accuracy:0.8709999918937683
Epoca 9 accuracy: 0.996666669845581
	 Test accuracy:0.8679999709129333
Epoca 10 accuracy: 0.9959999918937683
	 Test accuracy:0.8690000176429749
Epoca 11 accuracy: 0.996999979019165
	 Test accuracy:0.871999979019165
Epoca 12 accuracy: 0.9980000257492065
	 Test accuracy:0.878000020980835
Epoca 13 accuracy: 0.996999979019165
	 Test accuracy:0.8619999885559082
Epoca 14 accuracy: 0.9983333349227905
	 Test accuracy:0.870000